In [1]:
import json
import sys
from dotenv import load_dotenv, find_dotenv

sys.path.append('../../aisuite')

# Load from .env file if available
load_dotenv(find_dotenv())

True

In [2]:
import aisuite as ai
from aisuite.utils.tool_manager import ToolManager  # Import your ToolManager class

client = ai.Client()
tool_manager = ToolManager()

In [3]:
# Mock tool functions.
def get_current_temperature(location: str, unit: str):
    # Simulate fetching temperature from an API
    return {"location": location, "unit": unit, "temperature": 72}

tool_manager.add_tool(get_current_temperature)
messages = [{"role": "user", "content": "What is the current temperature in San Francisco in Celsius?"}]

In [4]:
# model = anthropic:claude-3-5-sonnet-20240620
# model = openai:gpt-4o
# model = mistral:mistral-large-latest
model = "aws:anthropic.claude-3-haiku-20240307-v1:0"
# model = "groq:llama-3.1-70b-versatile"
response = client.chat.completions.create(
    model=model, messages=messages, tools=tool_manager.tools())
print(response.choices[0])

Tool config: {'tools': [{'toolSpec': {'name': 'get_current_temperature', 'description': ' ', 'inputSchema': {'json': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': ''}, 'unit': {'type': 'string', 'description': ''}}, 'required': ['location', 'unit']}}}}]}
Received tools specification: [{'type': 'function', 'function': {'name': 'get_current_temperature', 'description': ' ', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': ''}, 'unit': {'type': 'string', 'description': ''}}, 'required': ['location', 'unit']}}}]
Formatted messages for Bedrock: [{'role': 'user', 'content': [{'text': 'What is the current temperature in San Francisco in Celsius?'}]}]
Bedrock response for tool use: {'ResponseMetadata': {'RequestId': '18e97a0d-73e5-4794-8e5e-e8d6534a7d90', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 17 Nov 2024 06:21:37 GMT', 'content-type': 'application/json', 'content-length': '317', 'connection': 'keep-al

In [5]:
response.choices[0].message

Message(content='', tool_calls=[ChatCompletionMessageToolCall(id='tooluse_POpRfVVZRKeGMKxXRemelQ', function=Function(arguments='{"location": "San Francisco", "unit": "Celsius"}', name='get_current_temperature'), type='function')], role='assistant', refusal=None)

In [6]:
from pprint import pprint
pprint(vars(response.choices[0].message))

{'content': '',
 'refusal': None,
 'role': 'assistant',
 'tool_calls': [ChatCompletionMessageToolCall(id='tooluse_POpRfVVZRKeGMKxXRemelQ', function=Function(arguments='{"location": "San Francisco", "unit": "Celsius"}', name='get_current_temperature'), type='function')]}


In [7]:
if response.choices[0].message.tool_calls:
    tool_results, result_as_message = tool_manager.execute_tool(response.choices[0].message.tool_calls)
    print(result_as_message)
    
    messages.append(response.choices[0].message) # Model's function call message
    messages.append(result_as_message[0])
    print(messages)

[{'role': 'tool', 'name': 'get_current_temperature', 'content': '{"location": "San Francisco", "unit": "Celsius", "temperature": 72}', 'tool_call_id': 'tooluse_POpRfVVZRKeGMKxXRemelQ'}]
[{'role': 'user', 'content': 'What is the current temperature in San Francisco in Celsius?'}, Message(content='', tool_calls=[ChatCompletionMessageToolCall(id='tooluse_POpRfVVZRKeGMKxXRemelQ', function=Function(arguments='{"location": "San Francisco", "unit": "Celsius"}', name='get_current_temperature'), type='function')], role='assistant', refusal=None), {'role': 'tool', 'name': 'get_current_temperature', 'content': '{"location": "San Francisco", "unit": "Celsius", "temperature": 72}', 'tool_call_id': 'tooluse_POpRfVVZRKeGMKxXRemelQ'}]


In [8]:
if response.choices[0].message.tool_calls:
    # Send the tool response back to the model
    final_response = client.chat.completions.create(
        model=model, messages=messages, tools=tool_manager.tools())

    # Output the final response from the model
    print(final_response.choices[0].message.content)

Tool config: {'tools': [{'toolSpec': {'name': 'get_current_temperature', 'description': ' ', 'inputSchema': {'json': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': ''}, 'unit': {'type': 'string', 'description': ''}}, 'required': ['location', 'unit']}}}}]}
Received tools specification: [{'type': 'function', 'function': {'name': 'get_current_temperature', 'description': ' ', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': ''}, 'unit': {'type': 'string', 'description': ''}}, 'required': ['location', 'unit']}}}]
Formatted messages for Bedrock: [{'role': 'user', 'content': [{'text': 'What is the current temperature in San Francisco in Celsius?'}]}, {'role': 'assistant', 'content': [{'toolUse': {'toolUseId': 'tooluse_POpRfVVZRKeGMKxXRemelQ', 'name': 'get_current_temperature', 'input': {'location': 'San Francisco', 'unit': 'Celsius'}}}]}, {'role': 'user', 'content': [{'toolResult': {'toolUseId': 'tooluse_POpRfVVZRK

In [ ]:
messages = [{"role": "user", "content": "What is the capital of California?"}]
response = client.chat.completions.create(
    model="openai:gpt-4o", messages=messages)
print(response.choices[0])